<a href="https://colab.research.google.com/github/IgorEisenlohr/Pyspark-first-contact/blob/main/Admission_prediction_with_pyspark_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=996c9e508d83b3e7f9f0b2431ee899be1b0d5769fb2318b97be11d5760d54e69
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 5.58 KiB | 5.58 MiB/s, done.


In [ ]:
#check the presence of dataset
!ls admission_dataset

Admission_Predict_Ver1.1.csv


In [ ]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv', header=True, inferSchema=True)

In [ ]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [ ]:
#get the no.of rows & columns
print((df.count(), len(df.columns)))

(500, 9)


In [ ]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [ ]:
#drop the unnecessary column
df = df.drop('Serial No')

In [ ]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values
print('Missing values:')
for i in df.columns:
  print(i+":", df[df[i].isNull()].count())

Missing values:
GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [ ]:
# correlation analysis
for col in df.columns:
  print(f'Correlation from {col} to Target Feature: {df.stat.corr("Chance of Admit", col)}')

Correlation from GRE Score to Target Feature: 0.8103506354632598
Correlation from TOEFL Score to Target Feature: 0.7922276143050823
Correlation from University Rating to Target Feature: 0.6901323687886892
Correlation from SOP to Target Feature: 0.6841365241316723
Correlation from LOR to Target Feature: 0.6453645135280112
Correlation from CGPA to Target Feature: 0.882412574904574
Correlation from Research to Target Feature: 0.5458710294711379
Correlation from Chance of Admit to Target Feature: 1.0


In [ ]:
# feature selection
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'CGPA'], 
                            outputCol='features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features', 'Chance of Admit')
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [ ]:
#build & train the model
lr = LinearRegression(featuresCol='features', labelCol='Chance of Admit')
lr_model = lr.fit(train)

In [ ]:
#get coefficients & intercept
print(f'intercept: {lr_model.intercept}')
print(f'coefficients: {lr_model.coefficients}')

intercept: -1.5710814368461845
coefficients: [0.002360568570305218,0.003460497006630509,0.13732369496674585]


In [ ]:
#get summary of the model
summary = lr_model.summary

In [ ]:
#print the rmse & r2 score
print(f'RMSE: {summary.rootMeanSquaredError}')
print(f'r2 score: {summary.r2}')

RMSE: 0.05907998462914046
r2 score: 0.8117046420227901


# TASK 6 : Evaluate & Save the Model

In [ ]:
#transform on the test data
pred = lr_model.transform(test)

In [ ]:
#display the predictions
pred.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [293.0,97.0,7.8]|           0.64| 0.5273581846370212|
|  [295.0,93.0,7.2]|           0.46|0.43584311677106236|
| [295.0,99.0,7.57]|           0.37| 0.5074158659485413|
| [295.0,99.0,7.65]|           0.57| 0.5184017615458809|
| [297.0,96.0,7.43]|           0.34| 0.4825301947739158|
| [297.0,99.0,7.81]|           0.54| 0.5450946898811706|
| [299.0,97.0,7.66]|           0.38| 0.5222962787635081|
|[299.0,100.0,7.88]|           0.68| 0.5628889826760837|
|  [300.0,99.0,6.8]|           0.36| 0.4134794636756729|
|[300.0,104.0,8.16]|           0.71| 0.6175421738635998|
| [301.0,96.0,7.56]|           0.54| 0.5098245494008138|
| [301.0,99.0,8.22]|           0.64| 0.6108396790987576|
|[301.0,106.0,8.47]|           0.57| 0.6693940818868573|
| [302.0,99.0,7.45]|           0.52| 0.5074610025446682|
| [302.0,99.0,7.97]|           

In [ ]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit',
                                metricName='r2')
print(f'r2 on test data: {round(evaluator.evaluate(pred),2)}')

r2 on test data: 0.79


In [ ]:
#save the model
lr_model.save("linear_regression_first_model")

In [ ]:
#load the model
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load("linear_regression_first_model")